# Digital Education & Learning analytics - Project part. 2

In [38]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('OutputTable.csv')
df.fillna(0, inplace=True)
df.query('SubmissionNumber > 0', inplace=True)
df['NVideoAndForum'] = df['NVideoEvents'].add(df['NForumEvents'])
df.head()
df.columns

Index(['ProblemID', 'UserID', 'SubmissionNumber', 'TimeStamp', 'TimeSinceLast',
       'Grade', 'GradeDiff', 'NVideoEvents', 'NForumEvents', 'NumberOfPosts',
       'TimeSpentOnForum', 'DurationOfVideoActivity', 'RewatchingScore',
       'NumberOfThreadCreated', 'LastVideoEvent', 'NumberVideoWatched',
       'NumberOfSlowPlay', 'NumberOfThreadViews', 'LastForumEvent',
       'NumberOfVideoInteractions', 'AverageVideoTimeDiffs', 'NumberOfUpvotes',
       'NVideoAndForum'],
      dtype='object')

In [14]:
def aggregate_df(df):
    aggregations = {
        'SubmissionNumber':{
            'TotalNumberOfSubmissions': 'last'
        },
        'TimeSinceLast':{
            'AvgTimeBwSubs': lambda x: np.log(np.mean(x))
        },
        'Grade':{ 
            'GradeDiff': lambda x: x.iloc[-1] - x.iloc[0]
        },
        'NVideoEvents':{
            'NVideoEvents': 'sum'
        },    
        'NForumEvents':{
            'NForumEvents': 'sum'
        },
        'NumberOfPosts':{
            'NumberOfPosts': 'sum'
        },
        'TimeSpentOnForum':{
            'TimeSpentOnForum': 'sum'
        },
        'DurationOfVideoActivity':{
            'DurationOfVideoActivity': 'sum'
        },
        'RewatchingScore':{
            'RewatchingScore': 'sum'
        },
        'NumberOfThreadCreated':{
            'NumberOfThreadCreated': 'sum'
        },
        'NumberVideoWatched':{
            'NumberVideoWatched': 'sum'
        },
        'NumberOfThreadViews':{
            'NumberOfThreadViews': 'sum'
        },
        'NumberOfVideoInteractions':{
            'NumberOfVideoInteractions': 'sum'
        },
        'NumberOfUpvotes':{
            'NumberOfUpvotes': 'sum'
        },
        'NVideoAndForum': {
            'TotalNVideoAndForum': 'sum',
            'ActivityRate': lambda x: np.sum(x !=0) / len(x)
        }
    }
    data = df.groupby(by=['UserID', 'ProblemID'], as_index=True).agg(aggregations)
    data.columns = data.columns.droplevel()
    data.query('TotalNumberOfSubmissions > 1', inplace=True)
    #data.query('TotalNVideoAndForum > 0', inplace=True)
    return data

In [16]:
data = aggregate_df(df)
data.head()

NVideoEvents  NumberOfPosts  \
UserID                           ProblemID                                
0009e7ff2e6bad5d80640eeb61cd6e0d 1                   0.0            0.0   
                                 4                   0.0            0.0   
                                 5                   0.0            0.0   
                                 11                  0.0            0.0   
                                 14                  0.0            0.0   

                                            TimeSpentOnForum  \
UserID                           ProblemID                     
0009e7ff2e6bad5d80640eeb61cd6e0d 1                       0.0   
                                 4                       0.0   
                                 5                       0.0   
                                 11                   2328.0   
                                 14                     51.0   

                                            NumberOfVideoInteractions  \
UserID                           ProblemID                              
0009e7ff2e6bad5d80640eeb61cd6e0d 1                                0.0   
                                 4                                0.0   
                                 5                                0.0   
                                 11                               0.0   
                                 14                               0.0   

                                             GradeDiff  NForumEvents  \
UserID                           ProblemID                             
0009e7ff2e6bad5d80640eeb61cd6e0d 1           12.500000           0.0   
                                 4          100.000000           0.0   
                                 5           13.333333           0.0   
                                 11          44.444444          30.0   
                                 14         100.000000          12.0   

                                            RewatchingScore  AvgTimeBwSubs  \
UserID                           ProblemID                                   
0009e7ff2e6bad5d80640eeb61cd6e0d 1                      0.0       4.595120   
                                 4                      0.0       4.276666   
                                 5                      0.0       5.034786   
                                 11                     0.0       6.482087   
                                 14                     0.0       7.757829   

                                            DurationOfVideoActivity  \
UserID                           ProblemID                            
0009e7ff2e6bad5d80640eeb61cd6e0d 1                              0.0   
                                 4                              0.0   
                                 5                              0.0   
                                 11                             0.0   
                                 14                             0.0   

                                            TotalNumberOfSubmissions  \
UserID                           ProblemID                             
0009e7ff2e6bad5d80640eeb61cd6e0d 1                                 2   
                                 4                                 2   
                                 5                                 3   
                                 11                               15   
                                 14                               22   

                                            NumberOfThreadCreated  \
UserID                           ProblemID                          
0009e7ff2e6bad5d80640eeb61cd6e0d 1                            0.0   
                                 4                            0.0   
                                 5                            0.0   
                                 11                           0.0   
                                 14                           0.0   

                  

In [25]:
y = data.GradeDiff.as_matrix()
X = data.drop('GradeDiff', axis=1).as_matrix()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Baseline model - linear regression

In [26]:
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [40]:
y_pred = reg.predict(X_test)
mean_squared_error(y_test, y_pred)

1242.7413713240019

In [39]:
scores = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error')
scores

array([-1270.95980092, -1410.0532485 , -1256.8083732 , -1280.21961331,
       -1328.30232063])